# Chapter 1 : 고성능을 위한 파이썬 이해하기
**High Performance Python (고성능 파이썬)** : 파이썬 성능 잠재력을 끌어내는 실용적인 개발 전략서, 한빛 미디어

### 이번 장에서 배울 내용
* 컴퓨터의 구성요소
* 컴퓨터 구성요소의 다양한 종류
* 파이썬에서 컴퓨터 구성요소를 추상화하는 방법
* 

---

![이미지!](img/cover_hpp.jpg)

### 들어가며 중 일부
누군가는 일련의 프로세스를 빠르게 실행해야 하고, 다른 누군가는 멀티 코어 아키텍쳐, 클러스터 또는 GPU를 제대로 활용하지 못해 애를 먹고 있을 것이다. 또 다른 누군가는 신뢰성을 잃지 않으면서도 주어진 예산 안에서 필요한 만큼의 프로세스를 사용할 수 있는 확장 가능한 시스템을 원한다. 어떤 사람은 코딩 실력의 부족함을 느끼기도 한다. 종종 다른 언어에서 차용한 기법이 예제에서 본 것만큼 자연스럽지 않은 경우도 있다.

* 컴퓨터의 저수준 동작방식을 이해하여 실제로 어떤 일이 일어나는 지 알 수 있다.
* 리스트와 튜플 : 기본적인 자료구조인 리스트와 튜플의 미묘한 의미적인 차이와 속도 차이
* 사전과 셋 : 중요한 자료구조인 사전과 셋의 메모리 할당 전략과 접근 알고리즘
* 이터레이터 : 이터레이터를 통해 데이터를 스트리밍하는 방법과 더 파이썬다운 코드를 작성하는 방법
* 순수 파이썬 방식의 접근 : 파이썬과 모듈을 효율적으로 사용하는 방법
* 행렬과 numpy : `numpy` 라이브러리를 고수처럼 사용하기
* 컴파일과 JIT : 빠른 처리를 위해 기계코드로 컴파일하기, 프로파일링 결과 반영하기
* 대이터를 효과적으로 옮기기 위한 동시적인 접근 방법
* multi-processing : 병렬 컴퓨팅을 위해 기본 라이브러리인 `multiprocessing` 모듈을 이용하는 다양한 방법, 효과적으로 `numpy` 행렬을 공유하기, 그리고 프로세스간 통신 (IPC) 비용과 장점
* 클러스터 컴퓨팅 : `multiprocessing`을 사용한 코드를 수정하여 로컬 또는 클러스터에서 실행하는 방법
* 메모리 아껴 쓰기 : 비싼 컴퓨터를 구입하지 않고 대용량 문제를 풀기 위한 접근법
* 현업에서 얻은 교훈 : 현업에서 고군분투했던 문제에서 얻은 교훈

### Python 2 vs Python 3
* 파이썬 3는 파이썬의 미래이고 모두가 파이썬 3로 갈아타는 중이다. 파이썬 2.7도 수년간 사용되긴 할 것이지만, **2020년까지만** 지원예정이다.
* 파이썬 3에서는 모든 변수가 **객체 (object)** 로 처리됩니다.
* 이외의 내부 변화가 있습니다.
    1. int 나누기 결과가 `float`
    2. `print`문 괄호 필수
    3. `str`과 `unicode`를 `str`로 통일함
    4. `long` 을 `int`로 통일
    5. `xrange` 지원하지 않는다.
    
   
    
* `__future__` 모듈

In [3]:
! git clone https://github.com/scari/high_performance_python

Cloning into 'high_performance_python'...
remote: Enumerating objects: 829, done.
remote: Total 829 (delta 0), reused 0 (delta 0), pack-reused 829
Receiving objects: 100% (829/829), 17.20 MiB | 1.19 MiB/s, done.
Resolving deltas: 100% (348/348), done.
